In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64

# Load data from CSV
df = pd.read_csv('aac_shelter_outcomes.csv')

# Remove '_id' field to avoid ObjectId issues
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Initialize the Dash app
app = JupyterDash(__name__)

# Load the Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Gregoria Ramirez'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '200px'}),
    html.Hr(),

    # Radio button filters
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mount'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset',
        labelStyle={'display': 'block'}
    ),
    
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable='single',
                         filter_action='native',
                         sort_action='native',
                         page_size=15,
                         style_table={'overflowX': 'auto'}),
    
    # chart and geolocation chart are side-by-side
    html.Div(className='row', style={'display' : 'flex'}, children=[
        html.Div(
            id='chart-id',
            className='col s12 m6', 
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
    ])
])

# Callback to update data table based on filter selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'water':
        filtered_df = df[(df['breed'].str.contains('Labrador Retriever', na=False)) & 
                         (df['outcome_type'] == 'Euthanasia') & 
                         (df['animal_type'] == 'Dog')]
    elif filter_type == 'mount':
        filtered_df = df[(df['outcome_type'] == 'Transfer') & 
                         (df['animal_type'] == 'Cat') & 
                         (df['sex_upon_outcome'].str.contains('Female', na=False)) & 
                         (df['age_upon_outcome_in_weeks'].between(52, 260))]
    elif filter_type == 'disaster':
        filtered_df = df[(df['animal_type'] == 'Dog') & 
                         (df['age_upon_outcome_in_weeks'].between(52, 260)) & 
                         (df['sex_upon_outcome'].str.contains('Male', na=False)) & 
                         (df['outcome_type'].isin(['Adoption', 'Transfer']))]
    else:
        filtered_df = df  # Reset case
    return filtered_df.to_dict('records')

# Callback to update the bar chart based on filtered data
@app.callback(
    Output('chart-id', 'children'),
    [Input('datatable-id', 'data'), Input('filter-type', 'value')]
)
def update_chart(data, filter_type):
    dff = pd.DataFrame(data)
    if dff.empty:
        return html.Div("No data available for chart.")
    
    animal_type_counts = dff['animal_type'].value_counts().reset_index()
    animal_type_counts.columns = ['animal_type', 'count']
    
    color_map = {
        'Cat': '#636EFA',
        'Dog': '#EF553B',
        'Other': '#00CC96',
        'Bird': '#AB63FA',
        'Livestock': '#000080'
        
    }
    
    fig = px.bar(animal_type_counts, x='animal_type', y='count',
                 title=f'Animal Type Counts for {filter_type}',
                 labels={'animal_type': 'Animal Type', 'count': 'Count'},
                 color='animal_type', color_discrete_map=color_map)
    
    return dcc.Graph(figure=fig)

# Callback to update the map based on the selected row in the table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or not index:
        return []

    dff = pd.DataFrame(viewData)
    row = index[0]

    lat = dff.iloc[row].get("location_lat", 30.75)
    lon = dff.iloc[row].get("location_long", -97.48)
    animal_name = dff.iloc[row].get("name", "Unknown")
    breed = dff.iloc[row].get("breed", "Unknown")

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[lat, lon], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[lat, lon],
                             children=[
                                 dl.Tooltip(breed),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(animal_name)
                                 ])
                             ])
               ])
    ]

if __name__ == '__main__':
    app.run_server(debug=True, port=8550)


Dash app running on http://127.0.0.1:8550/
